In [83]:
import numpy as np
import os
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [84]:
def load_data(folder):
    x_train = np.load(os.path.join(folder, 'x_train.npy'))
    y_train = np.load(os.path.join(folder, 'y_train.npy'))
    x_test = np.load(os.path.join(folder, 'x_test.npy'))
    y_test = np.load(os.path.join(folder, 'y_test.npy'))
    return x_train, y_train, x_test, y_test

In [85]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class LogisticRegression:
    def __init__(self, dim=2):
        rng = np.random.default_rng(seed=0)
        self.w = rng.normal(size=(dim, 1)) / np.sqrt(dim)
        self.b = np.zeros((1,))

    def predict(self, x, probs=False):
        # x - np.array размерности [N, dim]
        #     Массив входных признаков.
        assert x.shape[1] == self.w.shape[0], \
            "Размерность экземпляров данных не соответствует ожидаемой: " + \
            f"ожидалось x.shape[1]={self.w.shape[0]}, но было получено x.shape[1]={x.shape[1]}"

        x = x.dot(self.w) + self.b  # logits
        p = sigmoid(x)  # probabilities
        if probs:
            return p
        return np.array(p > 0.5).astype('int32')

    def fit(self, x, y, iters=1000, lr=0.01):
        # x - np.array размерности [N, dim]
        #     Массив входных признаков.
        # y - np.array размернсоти [N]
        #     Массив меток (правильных ответов).
        assert len(x) == len(y), \
            "Количество экземпляров в массиве X не равно количеству меток в массиве Y. " + \
            f"Полученные размеры: len(X) = {len(x)}, len(Y) = {len(y)}."
        assert x.shape[1] == self.w.shape[0], \
            "Размерность экземпляров данных не соответствует ожидаемой: " + \
            f"ожидалось x.shape[1]={self.w.shape[0]}, но было получено x.shape[1]={x.shape[1]}"
        # Алгоритм градиентного спуска.
        # Минимизируется бинарная кросс-энтропия.
        y = y.reshape(-1, 1)
        for i in range(iters):
            preds = self.predict(x, probs=True)
            self.w -= lr * np.mean(x.T.dot(preds - y), axis=1, keepdims=True)
            self.b -= lr * np.mean(preds - y, axis=0)
        return self

## 1. Применение логистической регрессии (несбалансированные данные)

### 1.1 Создание и обучение логистической регрессии

In [86]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [87]:
# Указание: производить нормализацию данных не нужно, это часть задания.
x_train, y_train, x_test, y_test = load_data('/content/drive/MyDrive/dataset1')

In [88]:
# Создайте модель логистической регрессии и обучите её, используя метод fit.
model = LogisticRegression(dim=x_train.shape[1])
model.fit(x_train, y_train)

In [89]:
# Получите предсказания на тестовой выборке и оцените точность модели,
# используя accuracy_score из пакета SciKit-Learn.
y_pred = model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9045


### 1.2 Анализ качества модели

In [90]:
# Допишите класс "глупого классификатора", что всегда предсказывает класс `0`.

class DummyClassifier:
    def __init__(self):
        print('Hello, brother🤪!')

    def predict(self, x):
        # x - numpy массив размерности [N, dim]
        # Должен возвращаться массив N предсказаний
        return np.zeros((x.shape[0],), dtype=int)

In [91]:
# Оцените точность "глупого классификатора", объясните результат.
dummy_model = DummyClassifier()
y_pred_dummy = dummy_model.predict(x_test)
accuracy_dummy = accuracy_score(y_test, y_pred_dummy)
print(f"My accuracy: {accuracy_dummy:.4f}")

Hello, brother🤪!
My accuracy: 0.9091


In [92]:
# Используйте дополнительные метрики (f1-score, recall, precision) из пакета sklearn для анализа "глупого классификатора".
f1_score_dummy = f1_score(y_test, y_pred_dummy, zero_division=0)
print(f"F1-score: {f1_score_dummy:.4f}")
precision_dummy = precision_score(y_test, y_pred_dummy, zero_division=0)
print(f"Precision: {precision_dummy:.4f}")
recall_dummy = recall_score(y_test, y_pred_dummy, zero_division=0)
print(f"Recall: {recall_dummy:.4f}")

F1-score: 0.0000
Precision: 0.0000
Recall: 0.0000


In [93]:
# Используя те же метрики, проанализируйте обученную вами модель логистической регрессии.
f1__score = f1_score(y_test, y_pred)
print(f"F1-score: {f1__score:.4f}")
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision:.4f}")
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall:.4f}")

F1-score: 0.4000
Precision: 0.4667
Recall: 0.3500


In [94]:
# Объясните результат, описав его комментариями в этой клетке.

# Можно заметить, что у модели логистической регрессии высокая accuracy и далеко не высокие, но не равные нулю, f1-score, recall, precision;
# а у глупого классификатора достаточно высокое значение accuracy, но все остальные метрики у него неприлично низкие, равные нулю.
# Такие результаты объясняются дисбалансом классов.
# F1-score, recall и precision дают более полное представление о качестве модели,
# т.к. accuracy оценивает, сколько предсказаний совпало с истинными метками (в случае глупого классификатора были предсказаны все нули,
# но в данных много нулей, поэтому точность у него такого высокая),
# precision - отношение количества правильных предсказаний одного класса к общему числу предсказаний этого класса
# recall показывает, сколько объектов одного класса было правильно предсказано,
# а f1-score гармоническое среднее между precision и recall.
# У глупого классификатора не было ни одного правильно предсказанного экземпляра класса "1", поэтому эти 3 метрики такие низкие.

### 1.3 Анализ набора данных

In [95]:
# Посчитайте количество экземпляров данных для каждого класса.
unique, counts = np.unique(y_train, return_counts=True)
number_of_class_instances = dict(zip(unique, counts))
print("Number of class instances in training set:", number_of_class_instances)

Number of class instances in training set: {0.0: 200, 1.0: 20}


In [96]:
# Предложите способ улучшения качества модели. Подсказка: добавление дубликатов в данные.
# Указание: не изменяйте тестовую выборку.

# Будем делать балансировку классов:
minority_class = min(number_of_class_instances, key=number_of_class_instances.get)
majority_class = max(number_of_class_instances, key=number_of_class_instances.get)

diff = number_of_class_instances[majority_class] - number_of_class_instances[minority_class]
minority_indices = np.where(y_train == minority_class)[0]
duplicated_indices = np.random.choice(minority_indices, diff, replace=True)

x_train_balanced = np.vstack([x_train, x_train[duplicated_indices]])
y_train_balanced = np.hstack([y_train, y_train[duplicated_indices]])

print("New number of class instances:", dict(zip(*np.unique(y_train_balanced, return_counts=True))))

New number of class instances: {0.0: 200, 1.0: 200}


In [97]:
# Создайте и обучите модель с использованием предложенных наработок.
model_new = LogisticRegression(dim=x_train_balanced.shape[1])
model_new.fit(x_train_balanced, y_train_balanced)

In [98]:
# Оцените качество новой модели, используя метрики из пакета sklearn.metrics.
# Указание: постарайтесь сбалансировать данные таким образом, чтобы новая модель была ощутимо лучше старой.
y_pred_new = model_new.predict(x_test)

accuracy_new = accuracy_score(y_test, y_pred_new)
print(f"Accuracy: {accuracy_new:.4f}")
f1_score_new = f1_score(y_test, y_pred_new)
print(f"F1-score: {f1_score_new:.4f}")
precision_new = precision_score(y_test, y_pred_new)
print(f"Precision: {precision_new:.4f}")
recall_new = recall_score(y_test, y_pred_new)
print(f"Recall: {recall_new:.4f}")

Accuracy: 0.9455
F1-score: 0.7143
Precision: 0.6818
Recall: 0.7500


## 2. Применение логистической регрессии (нелинейные данные)

In [99]:
x_train2, y_train2, x_test2, y_test2 = load_data('/content/drive/MyDrive/dataset2')

In [100]:
# Создайте и обучите модель но этом наборе данных.
model2 = LogisticRegression(dim=x_train2.shape[1])
model2.fit(x_train2, y_train2)

In [101]:
# Проанализируйте качество модели.
y_pred2 = model2.predict(x_test2)

accuracy2 = accuracy_score(y_test2, y_pred2)
print(f"Accuracy: {accuracy2:.4f}")
f1_score2 = f1_score(y_test2, y_pred2)
print(f"F1-score: {f1_score2:.4f}")
precision2 = precision_score(y_test2, y_pred2)
print(f"Precision: {precision2:.4f}")
recall2 = recall_score(y_test2, y_pred2)
print(f"Recall: {recall2:.4f}")

Accuracy: 0.5700
F1-score: 0.6195
Precision: 0.5147
Recall: 0.7778


In [102]:
# FEATURE ENGINEERING: попробуйте применить на исходных данных разные нелинейные функции (sin, tanh, ...).
# Объедините трансформированные данные с исходными (важно: количество экземпляров в x_train не должно увеличиться).
def nonlinear_transform(x):
    sin_x = np.sin(x)
    tanh_x = np.tanh(x)
    exp_x = np.exp(-x**2)
    x_transformed = np.hstack([x, sin_x, tanh_x, exp_x])
    return x_transformed

x_train_transformed = nonlinear_transform(x_train2)
x_test_transformed = nonlinear_transform(x_test2)

In [103]:
# Создайте и обучите модель с использованием наработок.
model_new2 = LogisticRegression(dim=x_train_transformed.shape[1])
model_new2.fit(x_train_transformed, y_train2)

In [104]:
# Оцените качество новой модели, используя метрики из пакета sklearn.metrics.
# Указание: постарайтесь добиться точности в 100%!
y_pred_new2 = model_new2.predict(x_test_transformed)

accuracy_new2 = accuracy_score(y_test2, y_pred_new2)
print(f"Accuracy: {accuracy_new2:.4f}")
f1_score_new2 = f1_score(y_test2, y_pred_new2)
print(f"F1-score: {f1_score_new2:.4f}")
precision_new2 = precision_score(y_test2, y_pred_new2)
print(f"Precision: {precision_new2:.4f}")
recall_new2 = recall_score(y_test2, y_pred_new2)
print(f"Recall: {recall_new2:.4f}")

Accuracy: 1.0000
F1-score: 1.0000
Precision: 1.0000
Recall: 1.0000


## 3. Доп. задания (любое на выбор, опционально)

### 3.1 'Упрощение' логистической регрессии

Сложность: легко.

In [105]:
"""
Модифицируйте класс логистической регрессии так, чтобы в нём не использовалась сигмоида.
То есть вывод о предсказанном классе должен делаться на основе значений "до сигмоиды".
Вспомогательная ссылка: https://en.wikipedia.org/wiki/Logit
"""

class LogisticRegression2:
    def __init__(self, dim=2):
        self.w = np.random.randn(dim, 1) / np.sqrt(dim)
        self.b = np.zeros((1,))

    def predict(self, x, probs=False):
        x = x.dot(self.w) + self.b
        #p = sigmoid(x)
        if probs:
            return x
        return np.array(x > 0).astype('int32')

    def fit(self, x, y, iters=1000, lr=0.01):
        y = y.reshape(-1, 1)
        for i in range(iters):
            preds = self.predict(x, probs=True)
            # градиентный спуск
            gradient_w = np.mean(x.T.dot(preds - y), axis=1, keepdims=True)
            gradient_b = np.mean(preds - y, axis=0)
            self.w -= lr * gradient_w
            self.b -= lr * gradient_b

In [106]:
# Перенесите обученные веса модели из пункта 1.3 в новую модель с модифицированным кодом
model3 = LogisticRegression2(dim=x_train_balanced.shape[1])

model3.w = model_new.w.copy()
model3.b = model_new.b.copy()

In [107]:
# Убедитесь, что предсказания модели с модифицированными кодом совпадают с предсказаниями
# модели из пункта 1.3
y_pred3 = model3.predict(x_test)

print(f"Предсказания совпадают: {np.array_equal(y_pred3, y_pred_new)}")

accuracy3 = accuracy_score(y_test, y_pred3)
print(f"Accuracy: {accuracy3:.4f}")
f1_score3 = f1_score(y_test, y_pred3)
print(f"F1-score: {f1_score3:.4f}")
precision3 = precision_score(y_test, y_pred3)
print(f"Precision: {precision3:.4f}")
recall3 = recall_score(y_test, y_pred3)
print(f"Recall: {recall3:.4f}")

Предсказания совпадают: True
Accuracy: 0.9455
F1-score: 0.7143
Precision: 0.6818
Recall: 0.7500


### 3.2 'Обобщение' логистической регрессии

Напишите многоклассовый классификатор. Обучите его на наборе данных ниже.

In [108]:
x_train3, y_train3, x_test3, y_test3 = load_data('/content/drive/MyDrive/dataset3')

<b>Ансамбль логистических регрессий.</b> Сложность: супергерой.

In [109]:
"""
Напишите класс, что инкапсулирует в себе `C` логистических регрессий,
где `C` - количество классов. i-ая логистическая регрессия производит
бинарную классификацию вида: все остальные классы и i-ый класс.
"""

class MulticlassLogisticRegression:
    def __init__(self, n_classes, dim):
        pass

    def predict(self, x):
        # x - numpy массив размерности [N, dim]
        # Возвращается массив целых чисел размерности [N],
        # где i-ый элемент обозначает номер класса для
        # i-го экземпляра данных в `x`.
        pass

    def fit(self, x, y):
        pass

In [110]:
# Создайте и обучите написанный классификатор. Оцените точность модели.


<b>Softmax классификатор.</b> Сложность: математический гений.

In [111]:
"""
Напишите класс классификатора, основанного на функции Softmax.
Алгоритм работы данного классификатора:
x - вектор (экземпляр данных) размерности dim.
W - матрица весов размерности [dim, n_classes].

Ответ классификатора формируется как:
logits = x * W - матричное умножение
p = softmax(logits)
class_id = argmax(p)

Для данного классификатора требуется модифицировать алгоритм обучения в методе fit.

Вспомогательные ресурсы:
https://en.wikipedia.org/wiki/Softmax_function
https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/
"""

class SoftmaxClassificator:
    def __init__(self, n_classes, dim):
        pass

    def predict(self, x):
        # x - numpy массив размерности [N, dim]
        # Возвращается массив целых чисел размерности [N],
        # где i-ый элемент обозначает номер класса для
        # i-го экземпляра данных в `x`.
        pass

    def fit(self, x, y):
        pass

In [112]:
# Создайте и обучите написанный классификатор. Оцените точность модели, посчитайте матрицу ошибок (выведите её с помощью matplotlib).


In [113]:
# Создайте и обучите написанный классификатор на наборе данных из задания 1 (опционально).
# Оцените точность модели, посчитайте матрицу ошибок (выведите её с помощью matplotlib).
